In [1]:
# Import Libraries
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# TorchVision Libraries
from torchvision import datasets
from torchvision.transforms import ToTensor



In [2]:
# Train and Test Dataset
train_dataloader = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download=True,
    target_transform = None
)
test_dataloader =  datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download=True,
)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|█████████████████████████████████████████████████| 9.91M/9.91M [00:27<00:00, 365kB/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|████████████████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 83.3kB/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|█████████████████████████████████████████████████| 1.65M/1.65M [00:04<00:00, 339kB/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|█████████████████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 959kB/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

